<a href="https://colab.research.google.com/github/targoons/ML/blob/master/MLAssignment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MsCA 31009 - Machine Learning and Predictive Analytics

## Assignment 7 - RNN

**Targoon Siripanichpong**

In [0]:
import sys
import os
import json
import pandas as pd
import numpy as np
import optparse


from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Running setup.py bdist_wheel for gputil ... - done
  Stored in directory: /root/.cache/pip/wheels/17/0f/04/b79c006972335e35472c0b835ed52bfc0815258d409f560108
Successfully built gputil
  Running setup.py bdist_wheel for humanize ... - done
  Stored in directory: /root/.cache/pip/wheels/69/86/6c/f8b8593bc273ec4b0c653d3827f7482bb2001a2781a73b7f44
Successfully built humanize
Gen RAM Free: 12.2 GB  | Proc size: 1.2 GB
GPU RAM Free: 10275MB | Used: 1166MB | Util  10% | Total 11441MB


**Mount Google Drive.**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


**Read data.**

In [0]:
mypath = '/content/gdrive/My Drive/UChicago/Machine Learning/Data/Raw'
import os
os.listdir(mypath)

['bottle.csv',
 'ProviderInfo.csv',
 'toxic_train.csv',
 'train.csv.zip',
 'default of credit card clients.xls',
 'dev-access.csv']

In [0]:
dev_access = pd.read_csv('/content/gdrive/My Drive/UChicago/Machine Learning/Data/Raw/dev-access.csv', engine='python', quotechar='|', header=None)

In [0]:
dev_access.head()

,0,1
0,"{""timestamp"":1502738402847,""method"":""post"",""qu...",0
1,"{""timestamp"":1502738402849,""method"":""post"",""qu...",0
2,"{""timestamp"":1502738402852,""method"":""post"",""qu...",0
3,"{""timestamp"":1502738402852,""method"":""post"",""qu...",0
4,"{""timestamp"":1502738402853,""method"":""post"",""qu...",0


In [0]:
dataset = dev_access.values

In [0]:
dataset.shape

(26773, 2)

In [0]:
X = dataset[:,0]
Y = dataset[:,1]

In [0]:
for index, item in enumerate(X):
  # Quick hack to space out json elements
  reqJson = json.loads(item, object_pairs_hook=OrderedDict)
  del reqJson['timestamp']
  del reqJson['headers']
  del reqJson['source']
  del reqJson['route']
  del reqJson['responsePayload']
  X[index] = json.dumps(reqJson, separators=(',', ':'))

In [0]:
tokenizer = Tokenizer(filters='\t\n', char_level=True)
tokenizer.fit_on_texts(X)

# we will need this later
num_words = len(tokenizer.word_index)+1
X = tokenizer.texts_to_sequences(X)

In [0]:
max_log_length = 1024
X_processed = sequence.pad_sequences(X, maxlen=max_log_length)

In [0]:
X_processed

array([[ 0,  0,  0, ...,  1, 19, 19],
       [ 0,  0,  0, ...,  1, 19, 19],
       [ 0,  0,  0, ...,  1, 19, 19],
       ...,
       [ 0,  0,  0, ...,  1, 19, 19],
       [ 0,  0,  0, ...,  1, 19, 19],
       [ 0,  0,  0, ...,  1, 19, 19]], dtype=int32)

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X_processed, Y, test_size=0.25, random_state=43)

In [0]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((20079, 1024), (20079,), (6694, 1024), (6694,))

In [0]:
num_words

63

In [0]:
model = Sequential()
model.add(Embedding(num_words, 32, input_length=max_log_length))
model.add(LSTM(units=64, recurrent_dropout=0.5))
model.add(Dense(units=1, activation='relu'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1024, 32)          2016      
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 26,913
Trainable params: 26,913
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_train, Y_train, epochs=3, batch_size=128, validation_split=0.25)

Train on 15059 samples, validate on 5020 samples
Epoch 1/3
15059/15059 [==============================] - 351s 23ms/step - loss: 0.7493 - acc: 0.5502 - val_loss: 0.6094 - val_acc: 0.6213
Epoch 2/3
15059/15059 [==============================] - 349s 23ms/step - loss: 0.5936 - acc: 0.6102 - val_loss: 0.5696 - val_acc: 0.5749
Epoch 3/3
15059/15059 [==============================] - 348s 23ms/step - loss: 0.6230 - acc: 0.5658 - val_loss: 0.5758 - val_acc: 0.5966


In [0]:
score = model.evaluate(X_test, Y_test, batch_size=128)

6694/6694 [==============================] - 52s 8ms/step


In [0]:
model2 = Sequential()
model2.add(Embedding(num_words, 32, input_length=max_log_length))
model2.add(Dropout(0.5))
model2.add(LSTM(units=64, recurrent_dropout=0.5))
model2.add(Dense(units=1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

In [0]:
model2.fit(X_train, Y_train, epochs=3, batch_size=128, validation_split=0.25)